**NEURAL NETWORKS**

GRUP 4: Arnau Alabau, Sergi Ubach, María Casals


In [1]:
# Import matplotlib, numpy and math
import matplotlib.pyplot as plt
import numpy as np
import math
import random

In [2]:
#Codi per treure la sigmoide

def sigmoide(aux):
    x = np.linspace(-100, 100, 200)
    z = aux/(1 + np.exp(-(aux - 0)))

    #plt.plot(x, z)
    #plt.xlabel("x")
    #plt.ylabel("Sigmoid(X)")

    #plt.show()
    #print("Z")
    #print(z)
    return z

### Class Neuron

Aquesta classe preten definir una neurona de la xarxa neuronal

*Atributs:*
- nInputs: numero d'entrades que te la neurona
- output: valor de sortida
- bias: "offset" necessari
- weight: array ambs els pesos que determinen cada entrada


*Funcions:*
- calcula: ha de fer el calcul (inputs*weight + bias ) *sigmoide 
    - DUBTE: No se com es la operacio en concret


In [3]:
class Neuron:
    
    def __init__(self, nInputs):
        
        #Atributs
        self.nInputs = 0
        self.output = 0
        self.bias = 0
        self.weight = []
        
        self.nInputs = nInputs
          
        self.bias = random.uniform(-self.nInputs, self.nInputs)

        for i in range (self.nInputs):
            self.weight.append(random.random())
            
    def calcula(self, inputs):
        #print('INPUTS \n')
        #print(inputs)
        #print('WEIGHT \n')
        #print(self.weight)
        #print('BIAS \n')
        #print(self.bias)
        
        aux = sum(np.multiply(inputs,self.weight)) + self.bias
        output = sigmoide(aux)
        
        return output
            

### Class Layer

Aquesta classe preten definir una capa de la xarxa neuronal

*Atributs:*
- nNeurons: numero de neurones que te la capa, coincideix amb el numero de outputs
- nInputs: numero d'entrades que te la capa


*Funcions:*
- calcula: ha de recorrer cada una de les neurones, passarli el vector d'entrada i obtenir la sortida de cada una


In [4]:
class Layer:
    
    def __init__(self, nNeurons, nInputs):
        
        #Atributs
        self.nNeurons = 0
        self.nInputs = 0 #num de neurones de la capa anterior, si es la 1a, es numInputs
        self.neurons = []
        #el num de neurones ve determinat pel numero de sortides, nOuputs = nNeurons
        
        self.nInputs = nInputs
        self.nNeurons = nNeurons
        for i in range (nNeurons):
            self.neurons.append(Neuron(nInputs))

            
    def calcula(self, inputs):
        output = []
        for i in range (self.nNeurons):
            output.append(self.neurons[i].calcula(inputs))
            #print(self.neurons[i].calcula(inputs))
            
        return output            

### Class NeuralNetwork

Aquesta classe preten definir la xarxa neuronal sencera sobre la que treballarem

*Atributs:*
- nLayers: numero de capes que te la xarxa
- nInputs: numero d'entrades que te la xarxa
- nOuputs: numero de sortides que te la xarxa


*Funcions:*
- calcula: no se que fa jaja


In [5]:
class NeuralNetwork:

    
    def __init__(self, nInputs, nLayers, neuronsInLayer):
        
        #Atributs
        self.nLayers = 0
        self.nInputs = 0
        self.neuronsInLayer = []
        self.layers = []

        self.nInputs = nInputs
        self.nLayers = nLayers
        self.neuronsInLayer = neuronsInLayer
        
        for i in range (self.nLayers):
             #crear layers
            if (i == 0) :
                self.layers.append(Layer(self.neuronsInLayer[i], self.nInputs))
            else:
                self.layers.append(Layer(self.neuronsInLayer[i], self.neuronsInLayer[i-1]))
        
        
    def calcula(self, inputs):
        
        for i in range (self.nLayers):
            if (i == 0) :
                aux = self.layers[i].calcula(inputs)
            else: 
                aux = self.layers[i].calcula(aux)
        return aux

In [6]:
def logicFunction(code):
    return (code[1]&(not code[2]))|(code[0]&code[1])

In [7]:
def calculaError(result, code):
    
    error = logicFunction(code) - result[0];
    
    return error

## Ask for data

In [8]:
nIndividuals = int(input("How many inidividuals per generation do you want? "))
nGenerations = int(input("How many generations do you want to try? "))


nInputs = int(input("How many inputs do you want in your Neural Network? "))
nLayers = int(input("How many layers do you want in your Neural Network? "))
neuronsInLayer = []

for i in range (nLayers):

    aux = int(input("How many neurons do you want in layer number "+ str(i+1)+"? "))
    neuronsInLayer.append(aux)

nLayers += 1
neuronsInLayer.append(1)

inputCode = [[0,0,0], [0,0,1], [0,1,0], [0,1,1], [1,0,0], [1,0,1], [1,1,0], [1,1,1]]

How many inidividuals per generation do you want? 10
How many generations do you want to try? 5
How many inputs do you want in your Neural Network? 3
How many layers do you want in your Neural Network? 2
How many neurons do you want in layer number 1? 4
How many neurons do you want in layer number 2? 5


## Generate X number of Neural Networks

In [9]:
def evaluate(nn):
    
    totalErrors = []
    for i in range (nIndividuals) :
        error = 0
        for j in range (len(inputCode) - 1):
            result = nn[i].calcula(inputCode[j])
            error += calculaError(result, inputCode[j])
        totalErrors.append(error)
    
    return totalErrors

In [10]:
def getParents(totalErrors, nn):
    mappedList = list(map(abs,totalErrors))
    inds = np.argsort(mappedList)
    print("INDS")
    print(inds)
    parent1 = nn[inds[0]]
    parent2 = nn[inds[1]]
    
    return parent1, parent2

In [11]:
def generateOffspring(parent1, parent2):
    
    offspring = []
    
    for i in range (nIndividuals - 2):
        offspring.append(NeuralNetwork(nInputs, nLayers, neuronsInLayer))
    
    offspring.append(parent1)
    offspring.append(parent2)
    
    for i in range(nIndividuals - 2):
        for j in range(nLayers):
            for m in range(offspring[i].neuronsInLayer[j]):
                p = random.uniform(0,1)
                if p <= 0.5:
                    #parent1
                    offspring[i].layers[j].neurons[m].bias = parent1.layers[j].neurons[m].bias
                if p > 0.5:
                    #parent2
                    offspring[i].layers[j].neurons[m].bias = parent2.layers[j].neurons[m].bias
                
                p = random.uniform(0,1)
                if p <= 0.5:
                    #parent1
                    offspring[i].layers[j].neurons[m].weight = parent1.layers[j].neurons[m].weight
                if p > 0.5:
                    #parent2
                    offspring[i].layers[j].neurons[m].weight = parent2.layers[j].neurons[m].weight 
    
                p = random.uniform(0,1)
                if p <= 0.1:
                    #mutation
                    print("MUTATION")
    
    return offspring
          

In [12]:
neuralNetworks = []

for i in range (nIndividuals):
    neuralNetworks.append(NeuralNetwork(nInputs, nLayers, neuronsInLayer))
    
totalErrors = evaluate(neuralNetworks)

print(totalErrors)
print(nIndividuals)

parent1, parent2 = getParents(totalErrors, neuralNetworks)

for i in range (nGenerations - 1):
    offspring = generateOffspring(parent1, parent2)
    totalErrors = evaluate(offspring)
    print("TOTAL ERRORS: ")
    print(totalErrors)
    parent1, parent2 = getParents(totalErrors, offspring)


[-114.99440709580364, -42.95442169349924, -52.1201170556693, -65.90128301519476, -121.57964557114649, -11.544782622234669, -94.12878284825305, -31.01603445620461, 2.8770108145498794, -42.4940559458462]
10
INDS
[8 5 7 9 1 2 3 6 0 4]
MUTATION
MUTATION
MUTATION
MUTATION
MUTATION
MUTATION
TOTAL ERRORS: 
[-22.51544334713981, -12.047782842157769, -25.4730086460811, -25.73563510057904, -10.473296995200386, -0.43774616198425276, -1.2944222484887231, -7.823039642507689, 2.8770108145498794, -11.544782622234669]
INDS
[5 6 8 7 4 9 1 0 2 3]
MUTATION
MUTATION
MUTATION
MUTATION
MUTATION
MUTATION
MUTATION
MUTATION
TOTAL ERRORS: 
[0.048754732476580465, -2.0755218635676536, 1.549320753187439, 0.34622697848146844, -4.112061721121723, -0.8909486579239286, -4.112061721121723, -2.5696563117950912, -0.43774616198425276, -1.2944222484887231]
INDS
[0 3 8 5 9 2 1 7 4 6]
MUTATION
MUTATION
MUTATION
MUTATION
MUTATION
MUTATION
MUTATION
MUTATION
TOTAL ERRORS: 
[0.34622697848146844, 0.048754732476580465, 0.3462269784